## Analisis de Datos: Tendencia de Victimas a Nivel Nacional a 1 año por Desapariciones

De acuerdo a los datos abiertos de la fiscalia se pueden agrupar la cantidad de victimas por año de hecho referentes al delito asociado que en nuestro caso es el secuestro y desapariciones

El objetivo de este notebook es entrenar una red neuronal para encontrar la tendencia y predecir 5 años a futuro el numero de victimas por año

### Cantidad de Victimas por Año

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# Conteo de Víctimas por año y municipio
dfSecuestros = pd.read_csv('ConteoVictimasDesapariciones.csv')
# Conteo de victimas por año de hecho
dfSecuestros[['ANIO_HECHO','TOTAL_VICTIMAS']].groupby('ANIO_HECHO').sum()

,TOTAL_VICTIMAS
ANIO_HECHO,
1968,1
1976,1
1977,1
1978,2
1979,5
1980,21
1981,34
1982,69
1983,54


### DataFrame 5 a 1 años

En este dataframe se representa lo siguiente:
- Input_0 -> Input_4 = Victimas por año en 5 años (ej: 1982 a 1986)
- Output_0 = Victimas en el año siguiente (ej: 1987)

In [4]:
# Conteo de Víctimas por año y municipio
dfVictimasAnios = pd.read_csv('DataSetDesapariciones5x1.csv')
dfVictimasAnios

,Input_0,Input_1,Input_2,Input_3,Input_4,Output_0
0,1,1,1,2,5,21
1,1,1,2,5,21,34
2,1,2,5,21,34,69
3,2,5,21,34,69,54
4,5,21,34,69,54,79
5,21,34,69,54,79,312
6,34,69,54,79,312,255
7,69,54,79,312,255,277
8,54,79,312,255,277,370
9,79,312,255,277,370,581


### Dividir el DataFrame en los conjuntos de prueba, validación y test

In [5]:
inputs = ['Input_{}'.format(i) for i in range(5)]
target_column = ['Output_0'] 

#predictors = list(set(list(df.columns))-set(target_column))
#df[predictors] = df[predictors]/df[predictors].max()
#df.describe()

X = dfVictimasAnios[inputs]
y = dfVictimasAnios['Output_0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(X_train.shape); print(X_test.shape)

(28, 5)
(13, 5)


### DataFrame a DataSet de Tensorflow

In [6]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(X,Y, shuffle=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((dict(X), Y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X))
    ds = ds.batch(batch_size)
    return ds

In [7]:
# Tamaño del numero de datos por iteración que se necesitan para entrenar y probar el algoritmo
batch_size = 4
train_ds = df_to_dataset(X_train,y_train, batch_size=batch_size)
test_ds = df_to_dataset(X_test,y_test, shuffle=False, batch_size=batch_size)

### Construyendo el Modelo

In [8]:
feature_columns = []

# Columnas Numericas
numericas = ['Input_{}'.format(i) for i in range(5)]
for header in numericas:
    feature_columns.append(feature_column.numeric_column(header))
    
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [25]:
model = tf.keras.Sequential([
    feature_layer,
    #layers.Dense(100, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',loss=losses.mean_squared_error,metrics=['mean_squared_error'])

model.fit(train_ds,epochs=20)
#model.summary()

Train for 7 steps
Epoch 1/20
7/7 [==============================] - 0s 30ms/step - loss: 16427207.6429 - mean_squared_error: 16427208.0000
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 15906148.0162 - mean_squared_error: 15906147.0000
Epoch 3/20
7/7 [==============================] - 0s 3ms/step - loss: 15438693.4107 - mean_squared_error: 15438694.0000
Epoch 4/20
7/7 [==============================] - 0s 3ms/step - loss: 15079729.2679 - mean_squared_error: 15079728.0000
Epoch 5/20
7/7 [==============================] - 0s 3ms/step - loss: 14715086.0000 - mean_squared_error: 14715086.0000
Epoch 6/20
7/7 [==============================] - 0s 3ms/step - loss: 14363784.7723 - mean_squared_error: 14363785.0000
Epoch 7/20
7/7 [==============================] - 0s 2ms/step - loss: 13978557.3438 - mean_squared_error: 13978559.0000
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 13636760.6964 - mean_squared_error: 13636761.0000
Epoch 9/20
7/7 [=====

In [26]:
pred= model.predict(test_ds)
print(np.sqrt(mean_squared_error(y_test,pred)))

3980.3445638499034
